In [199]:
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

## The ultimate goal of this project is to think of ways to improve collaboration throughout the company. 

#### Create a report that covers the following:  
####  1. Which departments are the most/least active?
####  2. Which employee has the most connections? 
####  3. Identify the most influential departments and employees.
####  4. Using the network analysis, in which departments would you recommend the HR team focus to boost collaboration?

In [200]:
# Import data from Datacamp downloaded file
employees = pd.read_csv(r'C:\Users\Nicholas.Xiang\OneDrive - Conrad Energy Limited\Desktop\Offline Work\Datacamp\Employee Network Analysis\data\employees.csv')
messages = pd.read_csv(r'C:\Users\Nicholas.Xiang\OneDrive - Conrad Energy Limited\Desktop\Offline Work\Datacamp\Employee Network Analysis\data\messages.csv')


In [201]:
# Merge both the dataframes into one 
# Merge the sender id to employee id
df1 = messages.merge(employees, left_on='sender', right_on='id', how='inner')
df1 = df1.rename(columns={'id': 'id_sender', 'department': 'department_sender', 'location': 'location_sender', 'age': 'age_sender'})
df1

# Merge the receiver id to employee id
df2 = df1.merge(employees, left_on='receiver', right_on='id', how='inner')
df2 = df2.rename(columns={'id': 'id_receiver', 'department': 'department_receiver', 'location': 'location_receiver', 'age': 'age_receiver'})
df2

,sender,receiver,timestamp,message_length,id_sender,department_sender,location_sender,age_sender,id_receiver,department_receiver,location_receiver,age_receiver
0,79,48,2021-06-02T05:41:34.000Z,88,79,Sales,France,33,48,IT,France,34
1,144,48,2021-06-05T11:19:34.000Z,33,144,Sales,US,50,48,IT,France,34
2,144,48,2021-06-05T11:35:09.000Z,85,144,Sales,US,50,48,IT,France,34
3,144,48,2021-06-05T11:39:57.000Z,69,144,Sales,US,50,48,IT,France,34
4,144,48,2021-06-07T08:09:32.000Z,28,144,Sales,US,50,48,IT,France,34
...,...,...,...,...,...,...,...,...,...,...,...,...
3507,1881,1079,2021-11-17T07:11:12.000Z,22,1881,Sales,Germany,57,1079,Engineering,UK,58
3508,1881,1079,2021-11-17T07:21:27.000Z,41,1881,Sales,Germany,57,1079,Engineering,UK,58
3509,1881,1079,2021-11-17T07:25:51.000Z,28,1881,Sales,Germany,57,1079,Engineering,UK,58
3510,1881,1079,2021-11-17T07:27:59.000Z,45,1881,Sales,Germany,57,1079,Engineering,UK,58


### Which departments are the most/least active?

In [202]:
# Here we aggregate how many times a person from the department sent a message (column which to aggregate doesn't matter since we only wanted count)
sender = df2.groupby(['department_sender']).agg({'sender': 'count'}).reset_index()
receiver = df2.groupby(['department_receiver']).agg({'receiver': 'count'}).reset_index()

sender_receiver = sender.merge(receiver, left_on='department_sender', right_on='department_receiver', how='inner').drop('department_receiver', axis=1).rename(columns={'department_sender': 'department'})
sender_receiver

,department,sender,receiver
0,Admin,857,797
1,Engineering,26,252
2,IT,49,249
3,Marketing,16,140
4,Operations,1013,845
5,Sales,1551,1229


In [203]:
# from plotly.graph_objs import *

# Colors to be used in the graph
colors = ['indianred'] * 6
colors[3] = 'lightblue'
colors[5] = 'darkblue'

colors2 = ['lightsalmon'] * 6
colors2[3] = 'lightblue'
colors2[5] = 'darkblue'




# fig = px.bar(
#              sender_receiver, x=sender_receiver.department, y=[sender_receiver.sender, sender_receiver.receiver], 
#              title = 'Amount of Messages Sent/Received by Department',
#              labels = {
#                        'value':'Number of Messages Sent/Received',
#                        'department':'Department',
#                        'variable': 'Legend'
#              },
#              barmode = "group",
#              text_auto=True
#             )
# fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
# fig.show()


fig = go.Figure()
fig.add_trace(go.Bar(
    x=sender_receiver.department,
    y=sender_receiver.sender,
    name='Sender',
    marker_color=colors,
    text = sender_receiver.sender,
    textposition="outside"
))
fig.add_trace(go.Bar(
    x=sender_receiver.department,
    y=sender_receiver.receiver,
    name='Receiver',
    marker_color=colors2,
    text = sender_receiver.receiver,
    textposition="outside"
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title = 'Amount of Messages Sent/Received by Department',
                  xaxis_title="Department",
                  yaxis_title="Number of Messages Sent/Received",
                  legend_title='Legend'
                  )
# fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
# fig.add_annotation(x='Marketing', y=200,
#             text="Least active",
#             showarrow=False,
#             yshift=100,
#             arrowhead=1)

# fig.add_annotation(x='Sales', y=1600,
#             text="Most active",
#             showarrow=False,
#             yshift=100,
#             arrowhead=1)

fig.show()

### Graph Analysis
##### From the graph above, we can see that the Sales department (highlighted in dark blue) sent and received the most messages, while the Marketing department (highlighted in light blue) sent and received the least amount of messages. We can say that the Sales department is the most active departments with their high volume of messages exchanged while the Marketing department is the least.

### Which employee has the most connections?

In [204]:
# To find employee with most connections, we need to make sure the sender and the receiver can't be repeated (since it would only counted as 'knowing' one person)
# So we need to filter out these rows by picking sender and receiver, message_length is a randomly picked column for us to count later
connection = df2[['sender', 'receiver', 'message_length']]
connection = connection.sort_values('message_length').drop_duplicates(subset=['sender', 'receiver'], keep='last')


# Count the number of (distinct) sender and receiver
connection_sender = connection.groupby(['sender']).agg({'message_length': 'count'}).sort_values(by=['message_length'], ascending=False).reset_index().rename(columns={'message_length': 'count'})
connection_receiver = connection.groupby(['receiver']).agg({'message_length': 'count'}).sort_values(by=['message_length'], ascending=False).reset_index().rename(columns={'message_length': 'count', 'receiver': 'sender'})

# Merge the two dataframes together on the employees' ID
connection_sender_receiver = connection_sender.merge(connection_receiver, on='sender', how='outer').rename(columns={'sender': 'id', 'count_x': 'messages_sent', 'count_y': 'messages_received'}).fillna(0)
connection_sender_receiver = connection_sender_receiver.sort_values(by='messages_sent', ascending=False)
connection_sender_receiver.sort_values(by='messages_sent', ascending=False)

# Need to change the employees' ID column to string because we need to treat it as an object/string
connection_sender_receiver.id = connection_sender_receiver.id.astype(str)


,id,messages_sent,messages_received
0,598,77.0,7.0
1,144,75.0,3.0
2,128,71.0,4.0
3,605,68.0,3.0
4,586,64.0,1.0
...,...,...,...
657,706,0.0,1.0
658,685,0.0,1.0
659,681,0.0,1.0
660,680,0.0,1.0


In [286]:

# Plot a scatter plot since I believe it's the best way to visualise the data here
fig = px.scatter(
                 connection_sender_receiver, 
                 x=connection_sender_receiver.messages_sent, 
                 y=connection_sender_receiver.messages_received, 
                 hover_name=connection_sender_receiver.id,
                 color=connection_sender_receiver.messages_sent,
                 title='Number of Messages Sent/Received from a Distinct Person',
                 labels={'messages_received': 'Number of Messages Received', 'messages_sent': 'Number of Messages Sent'}
                )

fig.update_traces(marker_size=8)

# Have to add annotations manually because plotly wont hide the overlapped labels if we do it manually and it's very messy
fig.add_annotation(x=77, y=7,
            text="598",
            showarrow=False,
            yshift=15)
fig.add_annotation(x=75, y=3,
            text="144",
            showarrow=False,
            yshift=15)
fig.add_annotation(x=71, y=4,
            text="128",
            showarrow=False,
            yshift=15)
fig.add_annotation(x=68, y=3,
            text="605",
            showarrow=False,
            yshift=15)
fig.add_annotation(x=64, y=1,
            text="586",
            showarrow=False,
            yshift=15)
fig.add_annotation(x=58, y=4,
            text="389",
            showarrow=False,
            yshift=15)
fig.add_annotation(x=50, y=5,
            text="509",
            showarrow=False,
            yshift=15)
fig.add_annotation(x=51, y=4,
            text="317",
            showarrow=False,
            yshift=-15)
fig.add_annotation(x=39, y=3,
            text="337",
            showarrow=False,
            yshift=15)
fig.add_annotation(x=33, y=6,
            text="260",
            showarrow=False,
            yshift=15)
fig.add_annotation(x=0, y=13,
            text="194",
            showarrow=False,
            yshift=15)
fig.add_annotation(x=0, y=11,
            text="32",
            showarrow=False,
            yshift=15)
fig.add_annotation(x=0, y=9,
            text="249",
            showarrow=False,
            yshift=15)

fig.show()


In [208]:
df2[df2.receiver == 706]


,sender,receiver,timestamp,message_length,id_sender,department_sender,location_sender,age_sender,id_receiver,department_receiver,location_receiver,age_receiver
1301,128,706,2021-07-01T02:13:35.000Z,73,128,Sales,France,47,706,Admin,US,45
1302,128,706,2021-07-01T05:28:23.000Z,23,128,Sales,France,47,706,Admin,US,45
1303,128,706,2021-07-01T06:14:22.000Z,85,128,Sales,France,47,706,Admin,US,45
1304,128,706,2021-07-07T04:44:30.000Z,51,128,Sales,France,47,706,Admin,US,45
1305,128,706,2021-07-07T04:46:56.000Z,19,128,Sales,France,47,706,Admin,US,45
1306,128,706,2021-07-07T05:45:13.000Z,63,128,Sales,France,47,706,Admin,US,45
1307,128,706,2021-07-07T07:31:35.000Z,35,128,Sales,France,47,706,Admin,US,45
1308,128,706,2021-07-07T19:57:04.000Z,15,128,Sales,France,47,706,Admin,US,45
1309,128,706,2021-07-08T00:42:31.000Z,26,128,Sales,France,47,706,Admin,US,45
1310,128,706,2021-07-08T01:02:41.000Z,86,128,Sales,France,47,706,Admin,US,45


In [209]:
test = df2.groupby('timestamp').agg({'receiver': 'count'}).reset_index()
test = test.sort_values(by=['timestamp'])
test

test = df2[df2.timestamp == '2021-11-24T05:04:57.000Z']
test

,sender,receiver,timestamp,message_length,id_sender,department_sender,location_sender,age_sender,id_receiver,department_receiver,location_receiver,age_receiver
2487,469,1629,2021-11-24T05:04:57.000Z,82,469,Operations,France,54,1629,Sales,Brasil,35
2490,469,1629,2021-11-24T05:04:57.000Z,75,469,Operations,France,54,1629,Sales,Brasil,35


In [210]:
gg2 = df2.groupby(['department_receiver', 'department_sender']).agg({'sender': 'count'})
gg2

sender
department_receiver department_sender        
Admin               Admin                 172
                    Engineering             4
                    IT                      8
                    Marketing               4
                    Operations            245
                    Sales                 364
Engineering         Admin                  33
                    Engineering             6
                    IT                      6
                    Marketing               2
                    Operations             91
                    Sales                 114
IT                  Admin                  48
                    Engineering             3
                    IT                      6
                    Operations             76
                    Sales                 116
Marketing           Admin                  19
                    IT                      3
                    Operations             40
                    Sales                  78
Operations          Admin                 287
                    Engineering             8
                    IT                     10
                    Marketing               2
                    Operations            198
                    Sales                 340
Sales               Admin                 298
                    Engineering             5
                    IT                     16
                    Marketing               8
                    Operations            363
                    Sales                 539